## Create json

In [ ]:
import requests
import base64
import json
import threading
import collections
from concurrent.futures import ThreadPoolExecutor

client_id = 'q4SKCJnxxpQ2qA0vGHL1CVRTCUrTLvqpUpj0Vvpl'
client_secret = 'G9hr2Dl5iWw7R5H0DXyic1fTxe4mwZZvDIuSSfICNudCQoyOOMxY6otlxW8KXUvhEDsQEiCycvoUAkjpW3o03fMABafFxayhc8zG0mwv9zauGcUBmjNAs9c8bVW3v70A'
auth_token = 'Basic ' + base64.b64encode("{}:{}".format(client_id,client_secret).encode()).decode('ascii')
headers = {'Authorization': auth_token}


def fetchID(index) :
    url = f'page={index}&category=Development&language=en&ordering=newest&ratings=4.5'
    page = requests.get(f'https://www.udemy.com/api-2.0/courses/?{url}', headers=headers)
    data = page.json()
    d1 = data['results']
    for i in range(len(d1)) :
        cid = d1[i]['id']
        IDlist.append(cid)
        
        
def multithreading(function, lst) :
    threads = []
    for i in range(len(lst)) :
        t = threading.Thread(target=function, args=(lst[i],))
        threads.append(t)
        t.start()
    for t in threads :
        t.join()

        
def fetchCourse(courseID) :
    url = f'https://www.udemy.com/api-2.0/courses/{courseID}/?fields%5bcourse%5d=@all'
    page = requests.get(url)
    course = page.json()
    print(courseID)
    return course

        
    
IDlist = []
pageNums = [i for i in range(1,53)]

multithreading(fetchID, pageNums)
print(f'Total number of courses fetched: {len(IDlist)}')
IDs = list(set(IDlist))
print(f'Total number of unique IDs: {len(IDs)}')
courses = {ID: None for ID in IDs}

pool = ThreadPoolExecutor(max_workers=100)
results = pool.map(fetchCourse, IDs)
final = []
for value in results :
    try:
        courses[value['id']] = value
    except ExecutorException :
        final.append(value['id'])
print(len(final))


with open('courses_total.json','w') as fh :
    json.dump(courses, fh, indent=3)  

## Create DB

In [ ]:
import json
import sqlite3

with open('courses_total.json') as fh :
    data = json.load(fh)   
        
conn = sqlite3.connect('courseDB.db')
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS CoursesDB") 
cur.execute('''CREATE TABLE CoursesDB(             
                   id INTEGER NOT NULL PRIMARY KEY,
                   title TEXT,
                   level_id INT,
                   payment_id INT,
                   price REAL,
                   rating REAL,
                   num_subscribers INT,
                   num_reviews INT,
                   suncategory_id INT,
                   created TEXT,
                   last_update_date TEXT,
                   contents TEXT,
                   requirements TEXT,
                   what_to_learn TEXT,
                   who_to_attend TEXT,
                   labels TEXT,
                   description TEXT,
                   headline TEXT,
                   url TEXT)''')
cur.execute("DROP TABLE IF EXISTS Payment")      
cur.execute("DROP TABLE IF EXISTS Level")      
cur.execute("DROP TABLE IF EXISTS Subcategory")      
cur.execute('''CREATE TABLE Payment(             
                   id INTEGER NOT NULL PRIMARY KEY UNIQUE,
                   payment TEXT UNIQUE ON CONFLICT IGNORE)''')    
cur.execute('''CREATE TABLE Level(             
                   id INTEGER NOT NULL PRIMARY KEY UNIQUE,
                   level TEXT UNIQUE ON CONFLICT IGNORE)''')   
cur.execute('''CREATE TABLE Subcategory(             
                   id INTEGER NOT NULL PRIMARY KEY UNIQUE,
                   subcategory TEXT UNIQUE ON CONFLICT IGNORE)''')  
for cid,dcts in data.items() :
    cur.execute('''INSERT INTO Payment (payment) VALUES (?)''', (dcts['is_paid'], ))
    cur.execute('SELECT id FROM Payment WHERE payment = ? ', (dcts['is_paid'], ))
    payment_id = cur.fetchone()[0]
    cur.execute('''INSERT INTO Level (level) VALUES (?)''', (dcts['instructional_level_simple'], ))
    cur.execute('SELECT id FROM Level WHERE level = ? ', (dcts['instructional_level_simple'], ))
    level_id = cur.fetchone()[0]
    cur.execute('''INSERT INTO Subcategory (subcategory) VALUES (?)''', (dcts['primary_subcategory']['title'], ))
    cur.execute('SELECT id FROM Subcategory WHERE subcategory = ? ', (dcts['primary_subcategory']['title'], ))
    subcategory_id = cur.fetchone()[0]
    cur.execute('''INSERT INTO CoursesDB
        (id, title, url, level_id, payment_id, price, rating, num_subscribers, num_reviews, suncategory_id, created, last_update_date, contents, requirements, what_to_learn, who_to_attend, labels, description, headline )
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', (cid, dcts['title'], dcts['url'],level_id, payment_id, dcts['price_detail']['amount'],dcts['rating'],dcts['num_subscribers'],dcts['num_reviews'], subcategory_id, dcts['created'], dcts['last_update_date'], dcts['content_info_short'], '; '.join(dcts['requirements_data']['items']), '; '.join(dcts['what_you_will_learn_data']['items']), '; '.join(dcts['who_should_attend_data']['items']), ', '.join([elem['label']['display_name'] for elem in dcts['course_has_labels']]), dcts['description'], dcts['headline'])) 

conn.commit()